# Exercise List - Strokes Data

Prof. Michel Fornaciali

Profa. Thanuci Silva

O objetivo da lista é praticar queries e operações com DataFrame. 

Abaixo está o dicionário de dados da tabela.

![a](https://miro.medium.com/max/649/1*3exwRyxaFZuhGB4hk-2K0Q.png)


### Cria uma tabela com DuckDB com os dados do arquivo `healthcare-dataset-stroke-data.csv` que estão no Blackboard

In [2]:
pip install duckdb

   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   ----- ---------------------------------- 1.6/11.4 MB 10.0 MB/s eta 0:00:01
   ------------ --------------------------- 3.7/11.4 MB 9.9 MB/s eta 0:00:01
   ------------------ --------------------- 5.2/11.4 MB 9.0 MB/s eta 0:00:01
   ------------------------- -------------- 7.3/11.4 MB 9.4 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.4 MB 8.5 MB/s eta 0:00:01
   ---------------------------------- ----- 10.0/11.4 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------  11.3/11.4 MB 8.0 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import duckdb
con = duckdb.connect()

In [ ]:
# Importing healthcare dataset

import pandas as pd

df = pd.read_table('healthcare-dataset-stroke-data.csv', header=0, sep=',')

con.register('df', df)

# Testing the connection and query execution

con.execute("select * from df limit 5").df()



,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


### Crie uma query SQL com a qual você consiga obter o número de pessoas com AVC por tipo de trabalho, coloque em ordem decrescente e renomeie a coluna com a contagem para `work_type_count`

In [13]:
query = """
select

work_type
,count(distinct id) as work_type_count

from df

where stroke = 1

group by 1

order by work_type_count desc
"""

con.execute(query).df()

,work_type,work_type_count
0,Private,149
1,Self-employed,65
2,Govt_job,33
3,children,2


### Crie uma query SQL com a qual você consiga obter o número total de pacientes por gênero e suas respectivas porcentagens em relação ao total de pacientes da base

In [15]:
query = """
select

gender
,count(distinct id) as pacient_count
,(count(distinct id) / (select count(distinct id) from df))*100 as percentage

from df

group by 1

"""

con.execute(query).df()

,gender,pacient_count,percentage
0,Male,2115,41.389432
1,Female,2994,58.590998
2,Other,1,0.019569


### Crie uma query SQL com a qual você consiga obter o número total de pacientes do sexo masculino que tiveram AVC e sua porcentagem em relação ao total da base

In [17]:
query = """
select

gender
,count(distinct id) as male_avc_count
,(count(distinct id) / (select count(distinct id) from df))*100 as total_percentage

from df

where stroke = 1 and gender = 'Male'

group by 1
"""

con.execute(query).df()

,gender,male_avc_count,total_percentage
0,Male,108,2.113503


### Crie uma query SQL com a qual você consiga obter o número total de pacientes que tiveram AVC por faixa etária

In [22]:
query = """
with age_grouped as (
select
id
, age
    ,case
        when age < 20 then '0-19'
        when age >= 20 and age < 30 then '20-29'
        when age >= 30 and age < 40 then '30-39'
        when age >= 40 and age < 50 then '40-49'
        when age >= 50 and age < 60 then '50-59'
        when age >= 60 and age < 70 then '60-69'
        when age >= 70 and age < 80 then '70-79'
        when age >= 80 and age < 90 then '80-89'
        when age >= 90 and age < 100 then '90-99'
        else '100+'
    end as age_group
,stroke
from df
)

select
age_group
,count(distinct id) as stroke_count

from age_grouped
where stroke = 1
group by 1
order by age_group

"""

con.execute(query).df()

,age_group,stroke_count
0,0-19,2
1,30-39,6
2,40-49,12
3,50-59,48
4,60-69,47
5,70-79,94
6,80-89,40
